In [63]:
import pandas as pd
import numpy as np

In [64]:
df = pd.read_csv('clicks.csv')

In [65]:
df

,user_id,group,click_day
0,8e27bf9a-5b6e-41ed-801a-a59979c0ca98,A,NaN
1,eb89e6f0-e682-4f79-99b1-161cc1c096f1,A,NaN
2,7119106a-7a95-417b-8c4c-092c12ee5ef7,A,NaN
3,e53781ff-ff7a-4fcd-af1a-adba02b2b954,A,NaN
4,02d48cf1-1ae6-40b3-9d8b-8208884a0904,A,Saturday
...,...,...,...
4993,1f6fd5aa-f664-4e7a-aa97-112572085691,C,NaN
4994,ad76e16a-b28f-4eab-9271-cb505e50fbcf,C,NaN
4995,c9254ed4-e47f-4c31-b541-67afb66ce273,C,NaN
4996,e734362b-42b9-4ec5-81e9-3b7ab326478e,C,NaN


In [66]:
df.head()

,user_id,group,click_day
0,8e27bf9a-5b6e-41ed-801a-a59979c0ca98,A,NaN
1,eb89e6f0-e682-4f79-99b1-161cc1c096f1,A,NaN
2,7119106a-7a95-417b-8c4c-092c12ee5ef7,A,NaN
3,e53781ff-ff7a-4fcd-af1a-adba02b2b954,A,NaN
4,02d48cf1-1ae6-40b3-9d8b-8208884a0904,A,Saturday


## Applying check for NaN function on dataframe columns

In [67]:
# Method 1
df['is_purchase'] = df.apply(lambda x: 'Purchase' if pd.notnull(x['click_day']) else 'No Purchase',axis = 1)

In [68]:
df

,user_id,group,click_day,is_purchase
0,8e27bf9a-5b6e-41ed-801a-a59979c0ca98,A,NaN,No Purchase
1,eb89e6f0-e682-4f79-99b1-161cc1c096f1,A,NaN,No Purchase
2,7119106a-7a95-417b-8c4c-092c12ee5ef7,A,NaN,No Purchase
3,e53781ff-ff7a-4fcd-af1a-adba02b2b954,A,NaN,No Purchase
4,02d48cf1-1ae6-40b3-9d8b-8208884a0904,A,Saturday,Purchase
...,...,...,...,...
4993,1f6fd5aa-f664-4e7a-aa97-112572085691,C,NaN,No Purchase
4994,ad76e16a-b28f-4eab-9271-cb505e50fbcf,C,NaN,No Purchase
4995,c9254ed4-e47f-4c31-b541-67afb66ce273,C,NaN,No Purchase
4996,e734362b-42b9-4ec5-81e9-3b7ab326478e,C,NaN,No Purchase


In [69]:
#Method 2
#df['ispurchase2'] = np.where(df['click_day'].isnull(), 'No purchase', 'Purchase')

In [70]:
df['group'].unique()

array(['A', 'B', 'C'], dtype=object)

In [82]:
purchase_counts = df.groupby(['group','is_purchase']).user_id.count().reset_index()

In [83]:
purchase_counts

,group,is_purchase,user_id
0,A,No Purchase,1350
1,A,Purchase,316
2,B,No Purchase,1483
3,B,Purchase,183
4,C,No Purchase,1583
5,C,Purchase,83


## Performing a Significance Test

In [84]:
from scipy.stats import chi2_contingency

In [85]:
contigency = [[316,1350],
              [183,1483],
              [83,1583]]

In [86]:
chi2, pvalue, dof, expected = chi2_contingency(contigency)

In [87]:
pvalue

2.4126213546684264e-35

In [98]:
t = lambda x : True if x < 0.05 else False    

In [99]:
is_significant = t(pvalue)

In [100]:
is_significant

True

In [91]:
# Confirmation

def is_sig(x):
    if x < 0.05:
        return True
    elif x > 0.05:
        return False
    else:
        return 'Equal'

In [92]:
is_sign = is_sig(pvalue)

In [93]:
is_sign

True

## Calculating Necessary Purchase Rates

In [101]:
num_visits = len(df['user_id'])

In [102]:
num_visits

4998

In [104]:
clicks_099 = 1000/0.99
clicks_199 = 1000/1.99
clicks_499 = 1000/4.99

In [108]:
p_clicks_099 = clicks_099 / num_visits
p_clicks_199 = clicks_199 / num_visits
p_clicks_499 = clicks_499 / num_visits

In [109]:
p_clicks_099

0.20210104243717691

In [110]:
p_clicks_199

0.10054272965467594

In [111]:
p_clicks_499

0.040096198800161346

## Performing a Significance Test II

In [112]:
from scipy.stats import binom_test

In [117]:
pvalueA = binom_test(316, n = 1666,p = p_clicks_099)
print(pvalueA)

0.2111287299402726


In [114]:
x = df.groupby(['group']).user_id.count().reset_index()

In [115]:
x

,group,user_id
0,A,1666
1,B,1666
2,C,1666


In [118]:
pvalueB = binom_test(183, n = 1666, p = p_clicks_199)

In [119]:
pvalueB

0.20660209246555486

In [120]:
pvalueC = binom_test(83, n = 1666, p = p_clicks_499)

In [121]:
pvalueC

0.045623672477172125

since pvalueC is less than 0.05 we can reject the null hypothesis and conclude that the probability of getting the expected purchase rate for 4.99 is statistically different than 0.040096